# The short notebook for submission 1
- Team Name: Random Forest Rangers
- Member 1: Andrés Zambrano / 122532
- Member 2: Mikael Aleksander Jansen Shahly / 544897

The Notebook will contain loading inn the data, all changes to the data etc to make the necessary training sets. 
It will then cover making the models and making a prediction

In [7]:
#importing necessary datasets
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns


#Setting max display options to avoid local crashes
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [8]:
Y_train_a = pd.read_parquet('../data/A/train_targets.parquet')
Y_train_b = pd.read_parquet('../data/B/train_targets.parquet')
Y_train_c = pd.read_parquet('../data/C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

Making dataset for location A

In [9]:
#Create the X dataset for location A
X_train_estimated_a["estimated"] = 1.0
X_test_estimated_a["estimated"] = 1.0
X_train_observed_a["estimated"] = 0.0
X_train_observed_a["date_calc"] = X_train_observed_a["date_forecast"].copy() #date calc column added to observed data so that observed and estimated has equal nr cols

X_train_estimated_a["train/test"] = "train"
X_train_observed_a["train/test"] = "train"
X_test_estimated_a["train/test"] = "test"

X_all_a= pd.concat([X_train_observed_a, X_train_estimated_a, X_test_estimated_a], ignore_index=True)
X_all_a.set_index('date_forecast', inplace=True)

#Create the complete train dataset for location A
train_all_a = pd.concat([X_train_observed_a.drop("train/test", axis=1), X_train_estimated_a.drop("train/test", axis=1)], ignore_index=True)
train_all_a.set_index('date_forecast', inplace=True)

#remove all non hourly values
train_all_a = train_all_a[train_all_a.index.minute == 0]

#Set the index as the date forecast for y aswell
Y_train_a.set_index('time', inplace=True)

#Concatenate with y data
train_all_a = pd.concat([train_all_a, Y_train_a], axis=1)

Making dataset for location B

In [10]:
#Combine all data from B
X_train_estimated_b["estimated"] = 1.0
X_test_estimated_b["estimated"] = 1.0
X_train_observed_b["estimated"] = 0.0
X_train_observed_b["date_calc"] = X_train_observed_b["date_forecast"].copy()

X_train_estimated_b["train/test"] = "train"
X_train_observed_b["train/test"] = "train"
X_test_estimated_b["train/test"] = "test"

X_all_b= pd.concat([X_train_observed_b, X_train_estimated_b, X_test_estimated_b], ignore_index=True)
X_all_b.set_index('date_forecast', inplace=True)

#Create the complete train dataset for location B
train_all_b = pd.concat([X_train_observed_b.drop("train/test", axis=1), X_train_estimated_b.drop("train/test", axis=1)], ignore_index=True)
train_all_b.set_index('date_forecast', inplace=True)
#remove all non hourly values
train_all_b = train_all_b[train_all_b.index.minute == 0]

#Set the index as the date forecast for y aswell
Y_train_b.set_index('time', inplace=True)

#Concatenate with y data
train_all_b = pd.concat([train_all_b, Y_train_b], axis=1)
#Drop the first row, as it is not in the X data
train_all_b.drop("2018-12-31 23:00:00", inplace=True)

Complete C dataset

In [11]:
#Combine all data from C
X_train_estimated_c["estimated"] = 1.0
X_test_estimated_c["estimated"] = 1.0
X_train_observed_c["estimated"] = 0.0
X_train_observed_c["date_calc"] = X_train_observed_c["date_forecast"].copy()
X_train_estimated_c["train/test"] = "train"
X_train_observed_c["train/test"] = "train"
X_test_estimated_c["train/test"] = "test"

X_all_c= pd.concat([X_train_observed_c, X_train_estimated_c, X_test_estimated_c], ignore_index=True)
X_all_c.set_index('date_forecast', inplace=True)

#Create the complete train dataset for location A
train_all_c = pd.concat([X_train_observed_c.drop("train/test", axis=1), X_train_estimated_c.drop("train/test", axis=1)], ignore_index=True)
train_all_c.set_index('date_forecast', inplace=True)
#remove all non hourly values
train_all_c = train_all_c[train_all_c.index.minute == 0]

#Set the index as the date forecast for y aswell
Y_train_c.set_index('time', inplace=True)

#Concatenate with y data
train_all_c = pd.concat([train_all_c, Y_train_c], axis=1)

train_all_c.drop("2018-12-31 23:00:00", inplace=True)

Make combined datasets for all locations

In [12]:
copy_a = X_all_a.copy()
copy_b = X_all_b.copy()
copy_c = X_all_c.copy()

copy_a["location"] = "A"
copy_b["location"] = "B"
copy_c["location"] = "C"
X_all = pd.concat([copy_a, copy_b, copy_c])

In [13]:
copy_a_train = train_all_a.copy()
copy_b_train = train_all_b.copy()
copy_c_train = train_all_c.copy()

copy_a_train["location"] = "A"
copy_b_train["location"] = "B"
copy_c_train["location"] = "C"

train_all = pd.concat([copy_a_train, copy_b_train, copy_c_train])

Create a test-set for all locations

In [14]:
X_test = X_all[X_all["train/test"] == "test"]
X_test = X_test.drop("train/test", axis=1)

Create a function for resampling to average X-values from the 15 minute intervals - called train_all_avg

In [15]:
def resample_df_no_categorical(df, hourly_df):
    indices_to_keep = hourly_df.index
    resampled_df = df.resample('1H').mean()
    resampled_df = resampled_df[resampled_df.index.isin(indices_to_keep)]
    return resampled_df

#use to create averages
train_all_avg_a = resample_df_no_categorical(X_all_a.drop(["train/test"], axis = 1), train_all_a)
train_all_avg_b = resample_df_no_categorical(X_all_b.drop(["train/test"], axis = 1), train_all_b)
train_all_avg_c = resample_df_no_categorical(X_all_c.drop(["train/test"], axis = 1), train_all_c)

In [16]:
#Add back locations
train_all_avg_a["location"] = "A"
train_all_avg_b["location"] = "B"
train_all_avg_c["location"] = "C"

#add in pv measurements
train_all_avg_c = pd.concat([train_all_avg_c, Y_train_c], axis=1)
train_all_avg_b = pd.concat([train_all_avg_b, Y_train_b], axis=1)
train_all_avg_a = pd.concat([train_all_avg_a, Y_train_a], axis=1)

train_all_avg = pd.concat([train_all_avg_a, train_all_avg_b,train_all_avg_c])

Make a function to remove the duplicates in the datasets

In [91]:
def remove_duplicates_over_threshold(df, threshold, col):
    threshold # Set your threshold here

    # Create a boolean mask to identify consecutive duplicates (non-zero values)
    mask = df[col].ne(df[col].shift()).cumsum()

    # Create a mask to identify rows that need to be kept (consecutive zeros, first in a group, or the first row of a group)
    rank_within_group = df.groupby(mask)[col].rank(method='first')
    keep_mask = (df[col] == 0) | (rank_within_group <= threshold - 1) | (mask != mask.shift())

    # Filter the DataFrame based on the keep_mask
    df_filtered = df[keep_mask]
    return df_filtered

train_all_avg = remove_duplicates_over_threshold(train_all_avg, 10, "pv_measurement").copy()

Make another function to remove all values under a certain thershold that is repeated (typically when sensor is covered in snow)

In [92]:
def remove_duplicates_over_threshold_under_val(df, col, threshold, val):
    # Define the threshold
    threshold

    # Create a boolean mask for values less than 1
    mask = df[col] == 0

    # Use cumulative sum to identify consecutive groups of values less than 1
    groups = (mask != mask.shift()).cumsum()

    # Filter out groups that don't meet the threshold
    valid_groups = groups[mask].value_counts() >= threshold
    valid_mask = groups.map(valid_groups.get).fillna(False)

    # Select rows that meet the criteria
    filtered_df = df[~valid_mask]
    return filtered_df

train_all_avg_no_lows = remove_duplicates_over_threshold_under_val(train_all_avg, "pv_measurement", 40, 1).copy()

#### Dropping rows containing only Nans

In [93]:
train_all.drop(train_all[train_all["absolute_humidity_2m:gm3"].isnull()].index, inplace=True)

In [94]:
train_all_avg.drop(train_all_avg[train_all_avg["absolute_humidity_2m:gm3"].isnull()].index, inplace=True)

In [95]:
train_all_avg_no_lows.drop(train_all_avg_no_lows[train_all_avg_no_lows["absolute_humidity_2m:gm3"].isnull()].index, inplace=True)

#### Drop snow denisity col as it contains almost only Nans

In [96]:
train_all = train_all.drop(["snow_density:kgm3"], axis=1)
train_all_avg = train_all_avg.drop(["snow_density:kgm3"], axis=1)
train_all_avg_no_lows = train_all_avg_no_lows.drop(["snow_density:kgm3"], axis=1)


X_test= X_test.drop(["snow_density:kgm3"], axis=1)

Remove all rows with Nans in target value

In [97]:
train_all = train_all.dropna(axis=0, subset=["pv_measurement"])
train_all_avg = train_all_avg.dropna(axis=0, subset=["pv_measurement"])
train_all_avg_no_lows = train_all_avg_no_lows.dropna(axis=0, subset=["pv_measurement"])

Preparing the fully to be used in the ML models

In [98]:
X_train_non_avg = train_all.drop(["pv_measurement", "date_calc"], axis=1)
y_train_non_avg= train_all[['pv_measurement']]

X_train_avg = train_all_avg.drop(["pv_measurement", "date_calc"], axis=1)
y_train_avg= train_all_avg[['pv_measurement']]

X_train_avg_no_lows = train_all_avg_no_lows.drop(["pv_measurement", "date_calc"], axis=1)
y_train_avg_no_lows = train_all_avg_no_lows[['pv_measurement']]

X_test = X_test.drop("date_calc", axis=1)

In [99]:
#Splitting out y-train values per location
y_train_a_avg = train_all_avg[train_all_avg["location"] == "A"][["pv_measurement"]]
y_train_b_avg = train_all_avg[train_all_avg["location"] == "B"][["pv_measurement"]]
y_train_c_avg = train_all_avg[train_all_avg["location"] == "C"][["pv_measurement"]]

y_train_a_non_avg = train_all[train_all["location"] == "A"][["pv_measurement"]]
y_train_b_non_avg = train_all[train_all["location"] == "B"][["pv_measurement"]]
y_train_c_non_avg = train_all[train_all["location"] == "C"][["pv_measurement"]]

y_train_a_avg_no_lows = train_all_avg_no_lows[train_all_avg_no_lows["location"] == "A"][["pv_measurement"]]
y_train_b_avg_no_lows = train_all_avg_no_lows[train_all_avg_no_lows["location"] == "B"][["pv_measurement"]]
y_train_c_avg_no_lows = train_all_avg_no_lows[train_all_avg_no_lows["location"] == "C"][["pv_measurement"]]

## We can now create the models and stack them

In [100]:
from catboost import Pool, CatBoostRegressor

The first models used in our stack

In [101]:
#Setting up avg / non-avg whole dataset

#Create a pool of data
train_pool_avg = Pool(X_train_avg, y_train_avg, cat_features=["location"])
test_pool_avg = Pool(X_test, cat_features=["location"]) 

train_pool_non_avg = Pool(X_train_non_avg, y_train_non_avg, cat_features=["location"])
test_pool_non_avg = Pool(X_test, cat_features=["location"]) 

#init models and fit them
catboost_model_avg = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100, random_seed=0)
catboost_model_avg.fit(train_pool_avg)

catboost_model_non_avg = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100, random_seed=0)
catboost_model_non_avg.fit(train_pool_non_avg)

0:	learn: 288.4237594	total: 357ms	remaining: 5m 56s
100:	learn: 95.0993860	total: 31.5s	remaining: 4m 40s
200:	learn: 85.7607295	total: 1m 4s	remaining: 4m 15s
300:	learn: 83.1566183	total: 1m 36s	remaining: 3m 44s
400:	learn: 81.0675620	total: 2m 10s	remaining: 3m 14s
500:	learn: 79.1934025	total: 2m 43s	remaining: 2m 42s
600:	learn: 77.3369244	total: 3m 17s	remaining: 2m 11s
700:	learn: 75.6892747	total: 3m 52s	remaining: 1m 39s
800:	learn: 73.9179017	total: 4m 27s	remaining: 1m 6s
900:	learn: 72.2539595	total: 5m 2s	remaining: 33.3s
999:	learn: 70.8529308	total: 5m 37s	remaining: 0us
0:	learn: 280.0059853	total: 271ms	remaining: 4m 30s
100:	learn: 99.0865828	total: 31.2s	remaining: 4m 37s
200:	learn: 90.2153171	total: 1m 3s	remaining: 4m 14s
300:	learn: 87.5726426	total: 1m 37s	remaining: 3m 45s
400:	learn: 85.6303550	total: 2m 10s	remaining: 3m 14s
500:	learn: 83.7731682	total: 2m 44s	remaining: 2m 44s
600:	learn: 82.4051164	total: 3m 19s	remaining: 2m 12s
700:	learn: 81.1314369	t

The second models used in our stack

In [104]:
# Add in dataset with lows removed
train_pool_avg_no_lows = Pool(X_train_avg_no_lows, y_train_avg_no_lows, cat_features=["location"])
test_pool = Pool(X_test, cat_features=["location"]) 

catboost_model_avg_no_lows = CatBoostRegressor(iterations=1000, depth=9, loss_function="LogCosh", verbose=100, random_seed=0)
catboost_model_avg_no_lows.fit(train_pool_avg_no_lows)

0:	learn: 318.9293862	total: 299ms	remaining: 4m 58s
100:	learn: 103.0052888	total: 29.2s	remaining: 4m 20s
200:	learn: 92.9501070	total: 59.5s	remaining: 3m 56s
300:	learn: 89.7083996	total: 1m 30s	remaining: 3m 29s
400:	learn: 87.6226020	total: 2m 1s	remaining: 3m 1s
500:	learn: 85.6117466	total: 2m 34s	remaining: 2m 34s
600:	learn: 83.8118371	total: 3m 8s	remaining: 2m 4s
700:	learn: 81.7809387	total: 3m 41s	remaining: 1m 34s
800:	learn: 80.0318201	total: 4m 14s	remaining: 1m 3s
900:	learn: 78.5463802	total: 4m 45s	remaining: 31.4s
999:	learn: 77.1956032	total: 5m 17s	remaining: 0us


The third models used in our stack
- Note this uses previoudsly found hyperparamaters with optuna. The code for finding these hyperparams is given at the end of the notebook

In [109]:
#create training location per location with prev found hyperparamaters focusing on 2020 summer (the old old ones)
params_a = {'iterations': 2511, 'learning_rate': 0.013387708538234228, 'depth': 9, 'min_data_in_leaf': 37, 'l2_leaf_reg': 9, 'bagging_temperature': 0.6037942407543951, 'random_strength': 0.7203046909719719, 'border_count': 792, 'rsm': 0.5142593972884377, "loss_function": "LogCosh"}
params_b = {'iterations': 1647, 'learning_rate': 0.008285667949530987, 'depth': 2, 'min_data_in_leaf': 7, 'l2_leaf_reg': 6, 'bagging_temperature': 0.638344903820208, 'random_strength': 0.9388444992830671, 'border_count': 434, 'rsm': 0.8347735297026142, "loss_function": "LogCosh"}
params_c = {'iterations': 2216, 'learning_rate': 0.02932152249119453, 'depth': 10, 'min_data_in_leaf': 21, 'l2_leaf_reg': 2, 'bagging_temperature': 0.8784102456931138, 'random_strength': 0.6901129919784297, 'border_count': 119, 'rsm': 0.8571604856673344, "loss_function": "LogCosh"}

X_train_a = X_train_non_avg[X_train_non_avg["location"] == "A"].drop("location", axis=1)
X_train_b = X_train_non_avg[X_train_non_avg["location"] == "B"].drop("location", axis=1)
X_train_c = X_train_non_avg[X_train_non_avg["location"] == "C"].drop("location", axis=1)

X_test_a = X_test[X_test["location"] == "A"].drop("location", axis=1)
X_test_b = X_test[X_test["location"] == "B"].drop("location", axis=1)
X_test_c = X_test[X_test["location"] == "C"].drop("location", axis=1)
          
train_pool_a = Pool(X_train_a, y_train_a_non_avg)
train_pool_b = Pool(X_train_b, y_train_b_non_avg)
train_pool_c = Pool(X_train_c, y_train_c_non_avg)


test_pool_a = Pool(X_test_a) 
test_pool_b = Pool(X_test_b) 
test_pool_c = Pool(X_test_c) 

catboost_model_a = CatBoostRegressor(**params_a, verbose = 400, random_seed=0)
catboost_model_b = CatBoostRegressor(**params_b, verbose = 400, random_seed=0)
catboost_model_c = CatBoostRegressor(**params_c, verbose = 400, random_seed=0)

catboost_model_a.fit(train_pool_a)
catboost_model_b.fit(train_pool_b)
catboost_model_c.fit(train_pool_c)

0:	learn: 624.9320516	total: 80.4ms	remaining: 3m 21s
400:	learn: 183.8413421	total: 34.2s	remaining: 3m
800:	learn: 169.7375507	total: 1m 8s	remaining: 2m 26s
1200:	learn: 155.6490973	total: 1m 43s	remaining: 1m 52s
1600:	learn: 146.4125240	total: 2m 18s	remaining: 1m 18s
2000:	learn: 139.1288148	total: 2m 48s	remaining: 42.9s
2400:	learn: 130.4735681	total: 3m 9s	remaining: 8.66s
2510:	learn: 129.1970689	total: 3m 15s	remaining: 0us
0:	learn: 96.2305278	total: 12.3ms	remaining: 20.3s
400:	learn: 45.4338503	total: 5.4s	remaining: 16.8s
800:	learn: 42.8962877	total: 11s	remaining: 11.6s
1200:	learn: 41.9358083	total: 16.8s	remaining: 6.22s
1600:	learn: 41.2727568	total: 22.7s	remaining: 651ms
1646:	learn: 41.2349579	total: 23.4s	remaining: 0us
0:	learn: 75.7193866	total: 50.2ms	remaining: 1m 51s
400:	learn: 15.2136583	total: 17.9s	remaining: 1m 21s
800:	learn: 11.5094546	total: 33.9s	remaining: 59.8s
1200:	learn: 8.6439264	total: 48.6s	remaining: 41.1s
1600:	learn: 7.0792931	total: 1m 

In [112]:
#Best params for 2021 avg loc by loc
params_avg_a = {'iterations': 2293, 'learning_rate': 0.008620188420007512, 'depth': 8, 'min_data_in_leaf': 18, 'l2_leaf_reg': 3, 'bagging_temperature': 0.4326298721660991, 'random_strength': 0.4658204470151926, 'border_count': 837, 'rsm': 0.7221138820493201, "loss_function": "LogCosh"}
params_avg_b = {'iterations': 2392, 'learning_rate': 0.02247702224978235, 'depth': 5, 'min_data_in_leaf': 94, 'l2_leaf_reg': 4, 'bagging_temperature': 0.7611940774372695, 'random_strength': 0.44589753796694487, 'border_count': 151, 'rsm': 0.783116168473357, "loss_function": "LogCosh"}
params_avg_c = {'iterations': 3025, 'learning_rate': 0.0017608361144341644, 'depth': 11, 'min_data_in_leaf': 18, 'l2_leaf_reg': 10, 'bagging_temperature': 0.7399543079455375, 'random_strength': 0.39726371700016055, 'border_count': 179, 'rsm': 0.6742668098873739, "loss_function": "LogCosh"}

X_train_avg_a = X_train_avg[X_train_avg["location"] == "A"].drop("location", axis=1)
X_train_avg_b = X_train_avg[X_train_avg["location"] == "B"].drop("location", axis=1)
X_train_avg_c = X_train_avg[X_train_avg["location"] == "C"].drop("location", axis=1)

X_test_a = X_test[X_test["location"] == "A"].drop("location", axis=1)
X_test_b = X_test[X_test["location"] == "B"].drop("location", axis=1)
X_test_c = X_test[X_test["location"] == "C"].drop("location", axis=1)
          
train_pool_avg_a = Pool(X_train_avg_a, y_train_a_avg)
train_pool_avg_b = Pool(X_train_avg_b, y_train_b_avg)
train_pool_avg_c = Pool(X_train_avg_c, y_train_c_avg)


test_pool_a = Pool(X_test_a) 
test_pool_b = Pool(X_test_b) 
test_pool_c = Pool(X_test_c) 

catboost_model_avg_a_best_hyper = CatBoostRegressor(**params_avg_a, verbose=400, random_seed=0)
catboost_model_avg_b_best_hyper = CatBoostRegressor(**params_avg_b, verbose=400, random_seed=0)
catboost_model_avg_c_best_hyper = CatBoostRegressor(**params_avg_c, verbose=400, random_seed=0)

catboost_model_avg_a_best_hyper.fit(train_pool_a)
catboost_model_avg_b_best_hyper.fit(train_pool_b)
catboost_model_avg_c_best_hyper.fit(train_pool_c)

0:	learn: 626.4702501	total: 29.8ms	remaining: 1m 8s
400:	learn: 194.8882584	total: 14.9s	remaining: 1m 10s
800:	learn: 182.6842082	total: 30.4s	remaining: 56.7s
1200:	learn: 175.2218005	total: 45.9s	remaining: 41.7s
1600:	learn: 169.9364448	total: 1m 1s	remaining: 26.6s
2000:	learn: 164.9970920	total: 1m 17s	remaining: 11.3s
2292:	learn: 161.3938650	total: 1m 29s	remaining: 0us
0:	learn: 95.3837887	total: 12.6ms	remaining: 30.2s
400:	learn: 37.0045121	total: 5.7s	remaining: 28.3s
800:	learn: 35.2342283	total: 11.7s	remaining: 23.3s
1200:	learn: 34.0703961	total: 17.8s	remaining: 17.6s
1600:	learn: 33.2272447	total: 24s	remaining: 11.8s
2000:	learn: 32.4425948	total: 30.2s	remaining: 5.91s
2391:	learn: 31.8306956	total: 36.5s	remaining: 0us
0:	learn: 77.3164904	total: 53.1ms	remaining: 2m 40s
400:	learn: 46.2400201	total: 23.9s	remaining: 2m 36s
800:	learn: 30.8360516	total: 48.8s	remaining: 2m 15s
1200:	learn: 24.0644687	total: 1m 13s	remaining: 1m 51s
1600:	learn: 20.9227284	total: 1

#### Creating the predictions from our models

In [113]:
pred_a = catboost_model_a.predict(test_pool_a)
pred_b = catboost_model_b.predict(test_pool_b)
pred_c = catboost_model_c.predict(test_pool_c)
pred_non_avg_per_location_best_hyper = np.concatenate((pred_a, pred_b, pred_c))

pred_non_avg_per_location_best_hyper[pred_non_avg_per_location_best_hyper < 0] = 0

In [114]:
pred_avg_a = catboost_model_avg_a_best_hyper.predict(test_pool_a)
pred_avg_b = catboost_model_avg_b_best_hyper.predict(test_pool_b)
pred_avg_c = catboost_model_avg_c_best_hyper.predict(test_pool_c)
pred_avg_per_location_best_hyper = np.concatenate((pred_avg_a, pred_avg_b, pred_avg_c))

pred_avg_per_location_best_hyper[pred_avg_per_location_best_hyper < 0] = 0

In [115]:
#Make predictions for non-summer
pred_avg = catboost_model_avg.predict(test_pool_avg)
pred_non_avg = catboost_model_non_avg.predict(test_pool_non_avg)
pred_avg[pred_avg < 0] = 0
pred_non_avg[pred_non_avg < 0] = 0

In [116]:
#Make predictions for non-summer avg with lows removed
pred_avg_no_lows = catboost_model_avg.predict(test_pool)
pred_avg_no_lows[pred_avg_no_lows < 0] = 0

In [125]:
pred_tot_stacked = pd.DataFrame((pred_avg + pred_non_avg + pred_non_avg_per_location_best_hyper + pred_avg_per_location_best_hyper + pred_avg_no_lows)/5)

predictions = pred_tot_stacked.clip(lower=0)

def replace_under_0_2(x):
    if x < 0.2:
        return 0
    else:
        return x
    
predictions = pred_tot_stacked.applymap(replace_under_0_2)

#### If wanted you can create a submission (as required for submitting to kaggle) below

In [126]:
def prepare_submission(predictions, X_test):
    index_df = X_test.index.to_frame()
    out_pd = pd.concat([index_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)
    out_pd = out_pd.rename(columns={0: 'prediction', 'date_forecast': 'time'})
    print(X_test.info())
    print(out_pd.info())
    out_pd['location'] = X_test['location'].reset_index(drop=True)
    out_pd.set_index('time', inplace=True)
    return out_pd

def merge_with_sample(out_pd):
    test = pd.read_csv('data/test.csv')
    test.time = pd.to_datetime(test.time)
    sample_submission = pd.read_csv('data/sample_submission.csv')
    test.set_index('time', inplace=True)
    
    merged_df = test.reset_index().merge(out_pd.reset_index(), on=['time', 'location'], how='left', suffixes=('_original', '_new'))
    merged_df['prediction_new'] = merged_df['prediction_new'].combine_first(merged_df['prediction_original'])
    merged_df.drop('prediction_original', axis=1, inplace=True)
    merged_df.rename(columns={'prediction_new': 'prediction'}, inplace=True)
    return sample_submission[['id']].merge(merged_df[['id', 'prediction']], on='id', how='left')


out_pd = prepare_submission(predictions, X_test)
sample_submission = merge_with_sample(out_pd)
sample_submission.to_csv('submission.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8640 entries, 2023-05-01 00:00:00 to 2023-07-03 23:45:00
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   absolute_humidity_2m:gm3        8640 non-null   float32
 1   air_density_2m:kgm3             8640 non-null   float32
 2   ceiling_height_agl:m            6243 non-null   float32
 3   clear_sky_energy_1h:J           8640 non-null   float32
 4   clear_sky_rad:W                 8640 non-null   float32
 5   cloud_base_agl:m                7690 non-null   float32
 6   dew_or_rime:idx                 8640 non-null   float32
 7   dew_point_2m:K                  8640 non-null   float32
 8   diffuse_rad:W                   8640 non-null   float32
 9   diffuse_rad_1h:J                8640 non-null   float32
 10  direct_rad:W                    8640 non-null   float32
 11  direct_rad_1h:J                 8640 non-null   float32
 12

## Code used to find hyper paramaters

In [ ]:
#simply change the dataframes in test_train_split function in order to get validation dataset for the avg values - used fof finding optimal hyperparams for avg values per location

def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_train_new_a, X_test_new_a, y_train_new_a, y_test_a = test_train_split(pd.concat([X_train_non_avg[X_train_non_avg["location"] == "A"].drop("location", axis=1), y_train_a_non_avg], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train_non_avg[X_train_non_avg["location"] == "B"].drop("location", axis=1), y_train_b_non_avg], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train_non_avg[X_train_non_avg["location"] == "C"].drop("location", axis=1), y_train_c_non_avg], axis=1))

In [ ]:
#Create a pool of data
train_pool_a = Pool(X_train_new_a, y_train_new_a)
train_pool_b = Pool(X_train_new_b, y_train_new_b)
train_pool_c = Pool(X_train_new_c, y_train_new_c)


test_pool_a = Pool(X_test_new_a) 
test_pool_b = Pool(X_test_new_b) 
test_pool_c = Pool(X_test_new_c) 

In [ ]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_a = CatBoostRegressor(**params, verbose=300)
    catboost_model_a.fit(train_pool_a)
    pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
    MAE_a = mean_absolute_error(y_test_a, pred_a)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)


#For location B
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_c = CatBoostRegressor(**params, verbose=400)
    catboost_model_c.fit(train_pool_c)
    pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))
    MAE_c = mean_absolute_error(y_test_c, pred_c)
    return MAE_c

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)


#For location C
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_b = CatBoostRegressor(**params, verbose=300)
    catboost_model_b.fit(train_pool_b)
    pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
    MAE_b = mean_absolute_error(y_test_b, pred_b)
    return MAE_b

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)